In [17]:
import feedparser
import hashlib
from threading import Timer
import time
from pandas import DataFrame
import datetime
import pandas as pd
import os.path
import csv
from random import randint

In [18]:
def write_dump(path, data):
    keys = data[0].keys()
    with open(path, 'wb') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(data)

In [19]:
def open_dump(path):
    if os.path.isfile(path):
        res = []
        with open(path) as csvfile:
            reader = csv.DictReader(csvfile)
            for r in reader:
                res.append(r)
            return res    

In [20]:
update_time = 60 # in seconds
dump_file = "news.csv"
set_columns = ['dt', 'ts', 'title', 'hash']

def scrap(hashes):
#    ts = int(time.time())
#    dt =  pd.to_datetime(datetime.datetime.now())
    data_temp = []
    feed_raw = feedparser.parse('https://fivethirtyeight.com/features/feed/')
    feed_entries = feed_raw['entries']
    dt = datetime.datetime.now()
    for f in feed_entries:
        
        title = f['title'].encode('utf-8')
        hashed_title = hashlib.md5(title).hexdigest()
        dt = dt + datetime.timedelta(hours=randint(1,12))
        ts = time.mktime(dt.timetuple())
        if not hashed_title in hashes:
            entry = {'dt': dt, 'ts' : ts, 'title' :  title, 'hash': hashed_title} 
            data_temp.append(entry)

    #df = DataFrame.from_records(data_temp, columns = set_columns) 
    return data_temp

def run_dump():
    data = open_dump(dump_file)
    if data is None or data == []:
        print "No CSV dump"
        # first start without csv
        res = scrap([])
        write_dump(dump_file, res)
    else:
        hashes = [e['hash'] for e in data]
        res = scrap(hashes)
        data = data + res
        write_dump(dump_file, data)
    Timer(update_time, run_dump).start()
 

In [21]:
run_dump()      

In [22]:
def read_dump(path):
    return pd.read_csv(path, index_col=0,parse_dates=[0])

In [23]:
def freq():
    df = read_dump(dump_file)
    group = df.groupby(pd.TimeGrouper(freq='1H'))

    group_counts = group.agg({'title':'count'})
    delta = group_counts.index.max() - group_counts.index.min()
    total = float(group_counts['title'].sum())
    total_hours =  delta.seconds / 60 / 60
    if total_hours == 0:
        frequency = 1
    else: 
        frequency = total / total_hours
    return frequency



In [24]:
print "news pear hour: ", abs(freq())

news pear hour:  3.42857142857
